### Suavizado para modelos de Lenguaje

Cuando entrenamos un modelo de lenguaje, puede suceder que, durante la fase de prueba, nos encontremos con palabras que están en nuestro vocabulario (es decir, no son palabras desconocidas), pero que aparecen en un contexto no visto. Por ejemplo, estas palabras podrían aparecer después de otra palabra con la que nunca se asociaron durante el entrenamiento. Para evitar que el modelo asigne una probabilidad de cero a estos eventos no vistos, es necesario redistribuir parte de la masa de probabilidad de eventos más frecuentes hacia estos eventos raros o no observados.

Esta técnica de redistribución de probabilidades se denomina suavizado o descuento (smoothing o discounting). En el estudio del suavizado para modelos de lenguaje, exploraremos varias técnicas: el suavizado de Laplace (también conocido como suavizado de adición-uno), el suavizado add-k, stupid backoff, y métodos más complejos como el suavizado de Kneser-Ney.

### Suavizado de Laplace 

El suavizado de Laplace es la forma más simple de suavizado que añade uno a todos los conteos de n-gramas antes de normalizarlos en probabilidades. Esto implica que incluso los n-gramas que no se observaron durante el entrenamiento recibirán una pequeña probabilidad en el modelo suavizado. 

El propósito es evitar que cualquier secuencia posible tenga una probabilidad de cero, lo que puede ser crítico en tareas como generación de texto y traducción automática.

Para un modelo de unigramas, la estimación de máxima verosimilitud (MLE) de la probabilidad de una palabra $w_i$ es su conteo $c_i$ normalizado por el número total de tokens de palabras $N$:

$$
P(w_i) = \frac{c_i}{N}
$$

El suavizado de Laplace modifica esta estimación al añadir uno a cada conteo:

$$
P_{\text{Laplace}}(w_i) = \frac{c_i + 1}{N + V}
$$

Aquí, $V$ es el tamaño del vocabulario. Añadiendo uno a cada conteo, aumentamos el numerador para cada palabra en el vocabulario. Para mantener la suma de probabilidades en $1$, también necesitamos ajustar el denominador, aumentando $N$ en $V$ para tener en cuenta estas $V$ observaciones adicionales.

**¿Por qué ajustamos el denominador?** Sin el ajuste del denominador, las probabilidades asignadas a los eventos existentes (vistos durante el entrenamiento) disminuirían en comparación con los eventos no vistos, distorsionando la distribución de probabilidad.

#### Ajuste de conteos con suavizado de Laplace

El suavizado de Laplace puede describirse en términos de un conteo ajustado $c^*$, que facilita la comparación directa con los conteos de máxima verosimilitud (MLE). El conteo ajustado para una palabra $w_i$ se define como:

$$
c^*_i = (c_i + 1) \frac{N}{N + V}
$$

Esta expresión ajusta el conteo original $c_i$ añadiendo uno y normalizando por el factor $\frac{N}{N + V}$. Este ajuste asegura que las probabilidades se mantengan correctamente distribuidas, de modo que la suma de todas las probabilidades es $1$.

Para convertir este conteo ajustado en una probabilidad, simplemente normalizamos por $N$:

$$
P_i^* = \frac{c^*_i}{N}
$$

#### Descuento relativo

El suavizado también puede verse como una forma de descuento (reducción) de los conteos observados. Al disminuir la probabilidad de eventos frecuentes, podemos redistribuir esa masa de probabilidad a eventos raros o no vistos. El descuento relativo $d_c$ es la proporción de los conteos ajustados con respecto a los conteos originales:

$$
d_c = \frac{c^*}{c}
$$

Esta relación permite ver el suavizado como un proceso que "descuenta" ciertos conteos para redistribuir la masa de probabilidad a otros eventos.

#### Suavizado de Laplace para bigramas

En el caso de bigramas, el suavizado de Laplace implica incrementar el conteo de cada bigrama observado en uno. La probabilidad de un bigrama suavizado se calcula como:

$$
P_{\text{Laplace}}(w_n | w_{n-1}) = \frac{C(w_{n-1}w_n) + 1}{C(w_{n-1}) + V}
$$

Aquí, $C(w_{n-1})$ es el conteo de la palabra anterior $w_{n-1}$, y $V$ es el tamaño del vocabulario. Esto asegura que incluso bigramas no observados reciban una probabilidad diferente de cero.

Los conteos ajustados para bigramas pueden reconstruirse mediante:

$$
c^*(w_{n-1}w_n) = \frac{[C(w_{n-1}w_n) + 1] \times C(w_{n-1})}{C(w_{n-1}) + V}
$$

Este ajuste permite analizar cuánto ha cambiado el algoritmo de suavizado los conteos originales.



### Ejemplos de Suavizado de Laplace

El suavizado de Laplace es una técnica sencilla pero poderosa para evitar que nuestro modelo de lenguaje asigne una probabilidad de cero a eventos no observados durante el entrenamiento. A continuación, se presentan algunos ejemplos que ilustran cómo aplicar las ecuaciones mencionadas.

#### Ejemplo 1: Suavizado de Laplace para unigramas

Supongamos un pequeño corpus con el siguiente conjunto de palabras: "gato", "perro", "pájaro", "pez", con los siguientes conteos:

- "gato": 3
- "perro": 2
- "pájaro": 1
- "pez": 0

El tamaño del corpus $N$ es la suma de los conteos: $N = 3 + 2 + 1 + 0 = 6$.

Supongamos que nuestro vocabulario $V$ contiene todas estas palabras, es decir, $V = 4$. Sin suavizado, las probabilidades de unigramas usando máxima verosimilitud (MLE) serían:

$$
P(\text{"gato"}) = \frac{3}{6} = 0.5
$$

$$
P(\text{"perro"}) = \frac{2}{6} \approx 0.333
$$

$$
P(\text{"pájaro"}) = \frac{1}{6} \approx 0.167
$$

$$
P(\text{"pez"}) = \frac{0}{6} = 0
$$

Observamos que "pez" tiene una probabilidad de cero, lo que podría ser problemático si "pez" aparece en los datos de prueba. Ahora, apliquemos el suavizado de Laplace a estas probabilidades.

Con suavizado de Laplace, añadimos $1$ a cada conteo y ajustamos el denominador sumando $V$:

$$
P_{\text{Laplace}}(w_i) = \frac{c_i + 1}{N + V}
$$

Para cada palabra:

$$
P_{\text{Laplace}}(\text{"gato"}) = \frac{3 + 1}{6 + 4} = \frac{4}{10} = 0.4
$$

$$
P_{\text{Laplace}}(\text{"perro"}) = \frac{2 + 1}{6 + 4} = \frac{3}{10} = 0.3
$$

$$
P_{\text{Laplace}}(\text{"pájaro"}) = \frac{1 + 1}{6 + 4} = \frac{2}{10} = 0.2
$$

$$
P_{\text{Laplace}}(\text{"pez"}) = \frac{0 + 1}{6 + 4} = \frac{1}{10} = 0.1
$$

Ahora, la palabra "pez" tiene una probabilidad distinta de cero, y la distribución de probabilidades está suavizada para que incluso las palabras no observadas tengan cierta probabilidad.

#### Ejemplo 2: Ajuste de conteos con suavizado de Laplace

Siguiendo con el mismo ejemplo, calculemos los conteos ajustados $c^*$ para la palabra "gato":

$$
c^*_{\text{"gato"}} = (3 + 1) \frac{N}{N + V} = 4 \times \frac{6}{10} = 2.4
$$

La probabilidad ajustada es entonces:

$$
P_{\text{ajustado}}(\text{"gato"}) = \frac{c^*_{\text{"gato"}}}{N} = \frac{2.4}{6} = 0.4
$$

Esto coincide con la probabilidad suavizada calculada anteriormente.

#### Ejemplo 3: Suavizado de Laplace para bigramas

Ahora, consideremos un modelo de bigramas con un corpus simple: "gato duerme", "perro ladra", "gato come". Los conteos de bigramas son:

- $ C(\text{"gato duerme"}) = 1$
- $C(\text{"perro ladra"}) = 1$
- $C(\text{"gato come"}) = 1$

El vocabulario $V$ contiene las palabras "gato", "duerme", "perro", "ladra", "come", entonces $V = 5$.

Queremos calcular la probabilidad suavizada del bigrama "gato come":

$$
P_{\text{Laplace}}(\text{"come"} | \text{"gato"}) = \frac{C(\text{"gato come"}) + 1}{C(\text{"gato"}) + V}
$$

Primero, necesitamos los conteos unigramas:

- $C(\text{"gato"}) = 2 $ (porque "gato" aparece dos veces como primera palabra en los bigramas)

Entonces,

$$
P_{\text{Laplace}}(\text{"come"} | \text{"gato"}) = \frac{1 + 1}{2 + 5} = \frac{2}{7} \approx 0.286
$$

Ahora calculemos los conteos ajustados para este bigrama:

$$
c^*(\text{"gato come"}) = \frac{[C(\text{"gato come"}) + 1] \times C(\text{"gato"})}{C(\text{"gato"}) + V} = \frac{[1 + 1] \times 2}{2 + 5} = \frac{4}{7} \approx 0.571
$$

Este conteo ajustado refleja cuánto ha cambiado el algoritmo de suavizado los conteos originales. Gracias al suavizado de Laplace, incluso los bigramas que no se observaron durante el entrenamiento ahora tienen una probabilidad diferente de cero, mejorando la capacidad del modelo para manejar eventos no vistos.


In [ ]:
# Suavizado de Laplace para Unigramas
def laplace_smoothing_unigram(corpus):
    # Conteo de palabras en el corpus
    word_counts = {}
    for word in corpus:
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1

    # Número total de tokens en el corpus
    N = sum(word_counts.values())

    # Tamaño del vocabulario
    V = len(word_counts)

    # Cálculo de las probabilidades suavizadas
    laplace_probabilities = {}
    for word, count in word_counts.items():
        # Aplicando la ecuación P_Laplace(w_i) = (c_i + 1) / (N + V)
        laplace_probabilities[word] = (count + 1) / (N + V)
    
    # Probabilidad para una palabra no vista
    laplace_probabilities['<UNK>'] = 1 / (N + V)
    
    return laplace_probabilities

# Ejemplo de uso
corpus = ["gato", "perro", "gato", "pájaro", "perro", "gato"]
laplace_prob_unigrams = laplace_smoothing_unigram(corpus)
print("Probabilidades de unigramas suavizadas con Laplace:")
for word, prob in laplace_prob_unigrams.items():
    print(f"P({word}) = {prob:.4f}")


In [ ]:
# Suavizado de Laplace para bigramas
def laplace_smoothing_bigram(corpus):
    # Conteo de bigramas y unigrams
    bigram_counts = {}
    unigram_counts = {}
    
    # Construir bigramas
    for i in range(len(corpus) - 1):
        bigram = (corpus[i], corpus[i+1])
        unigram = corpus[i]
        
        if bigram in bigram_counts:
            bigram_counts[bigram] += 1
        else:
            bigram_counts[bigram] = 1
        
        if unigram in unigram_counts:
            unigram_counts[unigram] += 1
        else:
            unigram_counts[unigram] = 1
    
    # Contar el último unigramo
    last_word = corpus[-1]
    if last_word in unigram_counts:
        unigram_counts[last_word] += 1
    else:
        unigram_counts[last_word] = 1
    
    # Tamaño del vocabulario
    V = len(unigram_counts)

    # Cálculo de las probabilidades suavizadas para bigramas
    laplace_probabilities = {}
    for bigram, bigram_count in bigram_counts.items():
        w_n_1 = bigram[0]
        # Aplicando la ecuación P_Laplace(w_n | w_n-1) = (C(w_n-1 w_n) + 1) / (C(w_n-1) + V)
        laplace_probabilities[bigram] = (bigram_count + 1) / (unigram_counts[w_n_1] + V)

    # Probabilidad para un bigrama no visto
    laplace_probabilities[('<UNK>', '<UNK>')] = 1 / (V * (V + 1))
    
    return laplace_probabilities

# Ejemplo de uso
corpus = ["gato", "duerme", "perro", "ladra", "gato", "come"]
laplace_prob_bigrams = laplace_smoothing_bigram(corpus)
print("\nProbabilidades de bigramas suavizadas con Laplace:")
for bigram, prob in laplace_prob_bigrams.items():
    print(f"P({bigram[1]} | {bigram[0]}) = {prob:.4f}")



In [ ]:
#  Suavizado de Laplace para trigramas

def laplace_smoothing_trigram(corpus):
    # Conteo de trigramas, bigramas y unigrams
    trigram_counts = {}
    bigram_counts = {}
    unigram_counts = {}
    
    # Construir trigramas
    for i in range(len(corpus) - 2):
        trigram = (corpus[i], corpus[i+1], corpus[i+2])
        bigram = (corpus[i], corpus[i+1])
        unigram = corpus[i]
        
        # Contar trigramas
        if trigram in trigram_counts:
            trigram_counts[trigram] += 1
        else:
            trigram_counts[trigram] = 1
        
        # Contar bigramas
        if bigram in bigram_counts:
            bigram_counts[bigram] += 1
        else:
            bigram_counts[bigram] = 1
        
        # Contar unigrams
        if unigram in unigram_counts:
            unigram_counts[unigram] += 1
        else:
            unigram_counts[unigram] = 1
    
    # Contar el último bigrama y unigramo restantes
    for i in range(len(corpus) - 1, len(corpus)):
        unigram = corpus[i]
        if unigram in unigram_counts:
            unigram_counts[unigram] += 1
        else:
            unigram_counts[unigram] = 1
        
    last_bigram = (corpus[-2], corpus[-1])
    if last_bigram in bigram_counts:
        bigram_counts[last_bigram] += 1
    else:
        bigram_counts[last_bigram] = 1
    
    # Tamaño del vocabulario
    V = len(unigram_counts)

    # Cálculo de las probabilidades suavizadas para trigramas
    laplace_probabilities = {}
    for trigram, trigram_count in trigram_counts.items():
        w_n_2_w_n_1 = (trigram[0], trigram[1])  # w_{n-2}w_{n-1}
        # Aplicando la ecuación P_Laplace(w_n | w_{n-2} w_{n-1}) = (C(w_{n-2}w_{n-1}w_n) + 1) / (C(w_{n-2}w_{n-1}) + V)
        laplace_probabilities[trigram] = (trigram_count + 1) / (bigram_counts[w_n_2_w_n_1] + V)

    # Probabilidad para un trigrama no visto
    laplace_probabilities[('<UNK>', '<UNK>', '<UNK>')] = 1 / (V * (V + 1) * (V + 2))
    
    return laplace_probabilities

# Ejemplo de uso
corpus = ["gato", "duerme", "perro", "ladra", "gato", "come", "globo", "vuela"]
laplace_prob_trigrams = laplace_smoothing_trigram(corpus)
print("\nProbabilidades de trigramas suavizadas con Laplace:")
for trigram, prob in laplace_prob_trigrams.items():
    print(f"P({trigram[2]} | {trigram[0]} {trigram[1]}) = {prob:.4f}")


La salida del código muestra las probabilidades suavizadas para cada trigrama en el corpus.

#### Ejercicios teóricos

1. **Comprensión de probabilidades con suavizado de Laplace:**
   - Supongamos un corpus de entrenamiento con las palabras: `["sol", "nube", "lluvia", "sol", "nube", "sol"]`.
   - Calcula manualmente las probabilidades de unigramas usando suavizado de Laplace. Utiliza las ecuaciones:
     $$
     P_{\text{Laplace}}(w_i) = \frac{c_i + 1}{N + V}
     $$
   - ¿Cuál es la probabilidad asignada a una palabra no vista como "viento"?

2. **Ajuste de conteos con suavizado de Laplace:**
   - Dado un conjunto de palabras con sus conteos $c_i$:
     - "gato": 3
     - "perro": 2
     - "ratón": 0
   - Utilizando la ecuación para ajustar los conteos:
     $$
     c^*_i = (c_i + 1) \frac{N}{N + V}
     $$
     Calcula los conteos ajustados $c^*_i$ para cada palabra. ¿Cómo cambian estos conteos en comparación con los conteos originales?

3. **Descuento relativo:**
   - Utilizando los conteos ajustados $c^*_i$ obtenidos en el ejercicio anterior, calcula el descuento relativo $d_c$ para cada palabra usando la ecuación:
     $$
     d_c = \frac{c^*}{c}
     $$
   - ¿Qué significa este descuento relativo en términos de la distribución de probabilidades?

4. **Probabilidad de bigrama con suavizado de Laplace:**
   - Dado el siguiente conjunto de bigramas y sus conteos:
     - $C(\text{"el gato"}) = 2$
     - $C(\text{"gato come"}) = 1$
     - $C(\text{"el perro"}) = 1$
   - Calcula la probabilidad del bigrama $P_{\text{Laplace}}(\text{"come"} | \text{"gato"})$ usando:
     $$
     P_{\text{Laplace}}(w_n | w_{n-1}) = \frac{C(w_{n-1}w_n) + 1}{C(w_{n-1}) + V}
     $$
   - ¿Cómo afecta el tamaño del vocabulario $V$ a la probabilidad suavizada?

5. **Aplicación de suavizado de Laplace a trigramas:**
   - Dado un conjunto de trigramas y sus conteos:
     - $C(\text{"el gato come"}) = 1$
     - $C(\text{"el perro duerme"}) = 1$
   - Utiliza la fórmula del suavizado de Laplace para trigramas:
     $$
     P_{\text{Laplace}}(w_n | w_{n-2}, w_{n-1}) = \frac{C(w_{n-2}w_{n-1}w_n) + 1}{C(w_{n-2}w_{n-1}) + V}
     $$
     Calcula la probabilidad del trigrama $P(\text{"come"} | \text{"el gato"})$.

6. Explica por qué el suavizado de Laplace evita que las palabras no vistas durante el entrenamiento tengan una probabilidad de cero. ¿Qué sucede con la distribución de probabilidades cuando se aplica el suavizado de Laplace a un conjunto de datos con un vocabulario muy grande?
7. Dado un corpus con las siguientes palabras y sus conteos: `{"gato": 4, "perro": 2, "pez": 0}`:
     - Calcula las probabilidades usando máxima verosimilitud (MLE).
     - Luego, aplica el suavizado de Laplace para calcular las probabilidades suavizadas.
     - Compara y analiza las diferencias entre ambos métodos. ¿Qué ventajas e inconvenientes tiene el suavizado de Laplace en este caso?
8. Usando la fórmula de ajuste de conteos $c^*$:
     $$
     c^*_i = (c_i + 1) \frac{N}{N + V}
     $$
     - Explica cómo esta fórmula afecta a los conteos originales. ¿Cómo cambia la distribución de probabilidades al aplicar este ajuste a un corpus con muchas palabras raras?
9. Explica cómo el suavizado de Laplace afecta a la probabilidad de bigramas en comparación con trigramas. ¿Es más efectivo el suavizado de Laplace en modelos de bigramas o trigramas? Justifica tu respuesta.
10. Discute cómo el suavizado de Laplace puede impactar tareas de NLP como la generación de texto y la traducción automática. ¿Hay situaciones en las que el suavizado de Laplace pueda ser contraproducente?

#### Ejercicios de codificación

1. **Suavizado de Laplace para trigramas con contenido real:**

    - Escribe una función que implemente el suavizado de Laplace para trigramas. Utiliza un texto real (por ejemplo, un fragmento de un libro) como corpus de entrada:
    ```python
      def laplace_smoothing_trigram(corpus):
    # Tu implementación aquí
    pass
    ```
    - Calcula las probabilidades suavizadas de trigramas en el texto. Identifica trigramas comunes y raros, y analiza cómo el suavizado de Laplace afecta sus probabilidades.
2. **Evaluación de probabilidades suavizadas:**
   - Escribe un script que compare las probabilidades suavizadas de un corpus usando diferentes tamaños de vocabulario $V$:
   ```python
   def evaluate_laplace_smoothing(corpus, vocab_sizes):
       # Tu implementación aquí
       pass
   ```
   - Experimenta con diferentes valores de $V$ y analiza cómo el tamaño del vocabulario afecta las probabilidades suavizadas en unigramas y bigramas.

3. **Suavizado de Laplace en un modelo de lenguaje simplificado:**
   - Crea un modelo de lenguaje simplificado que use el suavizado de Laplace para predecir la siguiente palabra en una secuencia basada en bigramas. La función debe predecir la palabra más probable dada una palabra anterior:
   ```python
   def predict_next_word(corpus, previous_word):
       # Tu implementación aquí
       pass
   ```
   - Prueba tu modelo con diferentes secuencias y analiza la calidad de las predicciones con y sin suavizado de Laplace.

4. **Comparación entre suavizado de Laplace y MLE:**
   - Implementa un script en Python que compare las probabilidades de unigramas, bigramas y trigramas calculadas con suavizado de Laplace y máxima verosimilitud (MLE).
   - Grafica las probabilidades obtenidas para visualizar las diferencias entre los dos métodos.

5. Después de implementar y probar el suavizado de Laplace para unigramas, bigramas y trigramas, reflexiona sobre las diferencias entre las probabilidades suavizadas y las probabilidades MLE. ¿Cómo afecta el suavizado al comportamiento del modelo en la práctica?

In [ ]:
## Tus respuestas

### Suavizado add-k

Una alternativa al suavizado de add-k es transferir un poco menos de la masa de probabilidad de los eventos vistos a los no vistos. En lugar de añadir 1 a cada conteo, añadimos un conteo fraccional $k$ (¿0.5? ¿0.05? ¿0.01?). Este algoritmo se llama por lo tanto suavizado de add-k.

$$P^*_{\text{Add-k}}(w_n | w_{n-1}) = \frac{C(w_{n-1}w_n) + k}{C(w_{n-1}) + kV}$$

El suavizado de add-k requiere que tengamos un método para elegir $k$. Esto puede hacerse, por ejemplo, optimizando en un conjunto de desarrollo o validación. Aunque el suavizado de `add-k` es útil para algunas tareas (incluida la clasificación de textos), resulta que todavía no funciona bien para el modelado de lenguajes, generando conteos con varianzas pobres y, a menudo, descuentos inapropiados.

#### Ejemplos Teóricos

#### Ejemplo 1: Suavizado Add-k para unigramas

Supongamos un corpus con los siguientes conteos unigramas:
- "gato": 3
- "perro": 2
- "ratón": 0

El tamaño del vocabulario $V = 3$ y el número total de tokens $N = 5$.

Para el suavizado add-k con $k = 0.5$:
1. **Probabilidad suavizada para "gato":**
   $$
   P^*_{\text{Add-0.5}}(\text{"gato"}) = \frac{3 + 0.5}{5 + 0.5 \times 3} = \frac{3.5}{6.5} \approx 0.538
   $$

2. **Probabilidad suavizada para "perro":**
   $$
   P^*_{\text{Add-0.5}}(\text{"perro"}) = \frac{2 + 0.5}{5 + 0.5 \times 3} = \frac{2.5}{6.5} \approx 0.385
   $$

3. **Probabilidad suavizada para "ratón":**
   $$
   P^*_{\text{Add-0.5}}(\text{"ratón"}) = \frac{0 + 0.5}{5 + 0.5 \times 3} = \frac{0.5}{6.5} \approx 0.077
   $$

#### Ejemplo 2: Suavizado Add-k para bigrama

Supongamos un corpus con los siguientes bigramas y sus conteos:
- $C(\text{"gato come"}) = 2$
- $C(\text{"perro come"}) = 1$

El tamaño del vocabulario $V = 3$. Queremos calcular la probabilidad suavizada para el bigrama "come" dado "gato" usando $k = 0.1$:

$$
P^*_{\text{Add-0.1}}(\text{"come"} | \text{"gato"}) = \frac{C(\text{"gato come"}) + k}{C(\text{"gato"}) + kV} = \frac{2 + 0.1}{2 + 0.1 \times 3} = \frac{2.1}{2.3} \approx 0.913
$$

In [ ]:
# Suavizado Add-k para unigramas
def add_k_smoothing_unigram(corpus, k):
    # Conteo de palabras en el corpus
    word_counts = {}
    for word in corpus:
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1

    # Número total de tokens en el corpus
    N = sum(word_counts.values())

    # Tamaño del vocabulario
    V = len(word_counts)

    # Cálculo de las probabilidades suavizadas
    add_k_probabilities = {}
    for word, count in word_counts.items():
        # Aplicando la ecuación P_Add-k(w_i) = (c_i + k) / (N + kV)
        add_k_probabilities[word] = (count + k) / (N + k * V)
    
    # Probabilidad para una palabra no vista
    add_k_probabilities['<UNK>'] = k / (N + k * V)
    
    return add_k_probabilities

# Ejemplo de uso
corpus = ["gato", "perro", "gato", "pájaro", "gato"]
k = 0.5
add_k_prob_unigrams = add_k_smoothing_unigram(corpus, k)
print("Probabilidades de unigramas suavizadas con Add-k (k=0.5):")
for word, prob in add_k_prob_unigrams.items():
    print(f"P({word}) = {prob:.4f}")


In [ ]:
#Suavizado add-k para bigrama
def add_k_smoothing_bigram(corpus, k):
    # Conteo de bigramas y unigrams
    bigram_counts = {}
    unigram_counts = {}

    # Construir bigramas
    for i in range(len(corpus) - 1):
        bigram = (corpus[i], corpus[i + 1])
        unigram = corpus[i]

        # Conteo de bigramas
        if bigram in bigram_counts:
            bigram_counts[bigram] += 1
        else:
            bigram_counts[bigram] = 1

        # Conteo de unigrams
        if unigram in unigram_counts:
            unigram_counts[unigram] += 1
        else:
            unigram_counts[unigram] = 1

    # Contar el último unigramo
    last_word = corpus[-1]
    if last_word in unigram_counts:
        unigram_counts[last_word] += 1
    else:
        unigram_counts[last_word] = 1

    # Tamaño del vocabulario
    V = len(unigram_counts)

    # Cálculo de las probabilidades suavizadas para bigramas
    add_k_probabilities = {}
    for bigram, bigram_count in bigram_counts.items():
        w_n_1 = bigram[0]  # w_{n-1}
        # Aplicando la ecuación P_Add-k(w_n | w_{n-1}) = (C(w_{n-1}w_n) + k) / (C(w_{n-1}) + kV)
        add_k_probabilities[bigram] = (bigram_count + k) / (unigram_counts[w_n_1] + k * V)

    # Probabilidad para un bigrama no visto
    add_k_probabilities[('<UNK>', '<UNK>')] = k / (V * (V + k))
    
    return add_k_probabilities

# Ejemplo de uso
corpus = ["gato", "come", "pájaro", "vuela", "gato", "duerme"]
k = 0.1
add_k_prob_bigrams = add_k_smoothing_bigram(corpus, k)
print("\nProbabilidades de bigramas suavizadas con Add-k (k=0.1):")
for bigram, prob in add_k_prob_bigrams.items():
    print(f"P({bigram[1]} | {bigram[0]}) = {prob:.4f}")


In [ ]:
# Suavizado add-k para trigramas

def add_k_smoothing_trigram(corpus, k):
    # Conteo de trigramas, bigramas y unigrams
    trigram_counts = {}
    bigram_counts = {}
    unigram_counts = {}
    
    # Construir trigramas
    for i in range(len(corpus) - 2):
        trigram = (corpus[i], corpus[i + 1], corpus[i + 2])
        bigram = (corpus[i], corpus[i + 1])
        unigram = corpus[i]
        
        # Contar trigramas
        if trigram in trigram_counts:
            trigram_counts[trigram] += 1
        else:
            trigram_counts[trigram] = 1
        
        # Contar bigramas
        if bigram in bigram_counts:
            bigram_counts[bigram] += 1
        else:
            bigram_counts[bigram] = 1
        
        # Contar unigrams
        if unigram in unigram_counts:
            unigram_counts[unigram] += 1
        else:
            unigram_counts[unigram] = 1

    # Contar el último unigramo
    last_word = corpus[-1]
    if last_word in unigram_counts:
        unigram_counts[last_word] += 1
    else:
        unigram_counts[last_word] = 1
    
    # Contar los últimos bigramas restantes
    for i in range(len(corpus) - 1, len(corpus)):
        last_bigram = (corpus[i - 1], corpus[i])
        if last_bigram in bigram_counts:
            bigram_counts[last_bigram] += 1
        else:
            bigram_counts[last_bigram] = 1
    
    # Tamaño del vocabulario
    V = len(unigram_counts)

    # Cálculo de las probabilidades suavizadas para trigramas
    add_k_probabilities = {}
    for trigram, trigram_count in trigram_counts.items():
        w_n_2_w_n_1 = (trigram[0], trigram[1])  # w_{n-2}w_{n-1}
        # Aplicando la ecuación P_Add-k(w_n | w_{n-2}w_{n-1}) = (C(w_{n-2}w_{n-1}w_n) + k) / (C(w_{n-2}w_{n-1}) + kV)
        add_k_probabilities[trigram] = (trigram_count + k) / (bigram_counts[w_n_2_w_n_1] + k * V)

    # Probabilidad para un trigrama no visto
    add_k_probabilities[('<UNK>', '<UNK>', '<UNK>')] = k / (V * (V + k))
    
    return add_k_probabilities

# Ejemplo de uso
corpus = ["el", "gato", "come", "pájaro", "vuela", "gato", "duerme", "el", "gato"]
k = 0.1
add_k_prob_trigrams = add_k_smoothing_trigram(corpus, k)
print("\nProbabilidades de trigramas suavizadas con Add-k (k=0.1):")
for trigram, prob in add_k_prob_trigrams.items():
    print(f"P({trigram[2]} | {trigram[0]} {trigram[1]}) = {prob:.4f}")


La salida de este código muestra las probabilidades suavizadas para los trigramas presentes en el corpus, asignando una probabilidad distinta de cero a los trigramas, incluso si no han sido observados durante el entrenamiento.

#### Ejercicios teóricos

1. **Comparación entre suavizado de Laplace y Add-k:**
   - Dado un conjunto de palabras con los siguientes conteos: 
     - "gato": 5
     - "perro": 3
     - "ratón": 0
   - Supón que el tamaño del vocabulario $V = 3$ y $N = 8$. Calcula las probabilidades suavizadas usando:
     - **Suavizado de Laplace**:
       $$
       P_{\text{Laplace}}(w_i) = \frac{c_i + 1}{N + V}
       $$
     - **Suavizado add-k** con \( k = 0.5 \):
       $$
       P^*_{\text{Add-k}}(w_i) = \frac{c_i + k}{N + kV}
       $$
   - Compara las probabilidades obtenidas con ambos métodos y discute las diferencias. ¿Qué sucede con la probabilidad asignada a la palabra "ratón"?

2. **Elección del valor $k$ en suavizado Add-k:**
   - Explica por qué es importante elegir el valor adecuado de $k$ en el suavizado **add-k**. ¿Qué sucede si $k$ es demasiado grande o demasiado pequeño? ¿Cómo afecta esto a la distribución de probabilidades?
   - Sugiere un método para elegir el valor óptimo de $k$ utilizando un conjunto de validación. ¿Qué métricas utilizarías para evaluar la calidad del suavizado?

3. **Aplicación en modelos de lenguaje:**
   - Discute por qué el suavizado **add-k** puede no ser efectivo para el modelado de lenguajes en comparación con otras técnicas de suavizado más avanzadas. ¿Qué problemas podrían surgir al usar suavizado **add-k** en modelos de lenguaje n-grama de mayor orden (bigramas, trigramas, etc.)?

4. **Impacto del suavizado en la clasificación de textos:**
   - El suavizado **add-k** se ha utilizado en tareas de clasificación de textos. Explica cómo el suavizado puede afectar la clasificación de documentos cuando se utilizan modelos de n-gramas. ¿En qué casos podría el suavizado **add-k** ser beneficioso para la clasificación de textos?

5. **Evaluación de suavizado en conjuntos de datos desbalanceados:**
   - Dado un conjunto de datos desbalanceado en el que algunas palabras ocurren con mucha más frecuencia que otras, ¿cómo afectaría el suavizado **add-k** a la probabilidad de palabras raras? ¿El suavizado de Laplace sería más o menos efectivo en este escenario?

#### Ejercicios prácticos

1. **Implementación del suavizado Add-k para unigramas y comparación con Laplace:**
   - Implementa una función en Python para calcular las probabilidades suavizadas usando el suavizado **add-k** para unigramas y compárala con el suavizado de Laplace:
   ```python
   def add_k_smoothing_unigram(corpus, k):
       # Tu implementación aquí
       pass

   def laplace_smoothing_unigram(corpus):
       # Implementación del suavizado de Laplace
       pass
   ```
   - Prueba ambas funciones con el siguiente corpus: `["gato", "perro", "gato", "ratón", "gato"]`.
   - Compara las probabilidades generadas por ambos métodos y analiza cómo el valor de $k$ afecta a los resultados.

2. **Optimización del valor $k$ usando validación cruzada:**
   - Implementa un script en Python que utilice validación cruzada para encontrar el valor óptimo de $k$ para un conjunto de datos dado. La función debe maximizar la precisión de las probabilidades suavizadas:
   ```python
   def find_optimal_k(corpus, k_values):
       # Tu implementación aquí
       pass
   ```
   - Utiliza esta función con un rango de valores de $k$ ( $k = [0.01, 0.05, 0.1, 0.5]$) para encontrar el valor óptimo que minimiza el error en un conjunto de validación.

3. **Implementación del suavizado Add-k para bigrama:**
   - Escribe una función en Python para calcular las probabilidades suavizadas usando el suavizado **add-k** para bigramas:
   ```python
   def add_k_smoothing_bigram(corpus, k):
       # Tu implementación aquí
       pass
   ```
   - Usa la función con el siguiente corpus: `["el", "gato", "come", "ratón", "el", "perro", "duerme"]` y compara los resultados con el suavizado de Laplace. ¿Qué cambios observas en las probabilidades de bigramas?

4. **Análisis de sensibilidad del valor $k$:**
   - Implementa un experimento que grafique la variación de las probabilidades suavizadas con diferentes valores de $k$ en un conjunto de bigramas. Utiliza las funciones implementadas para suavizado **add-k** y suavizado de Laplace:
   ```python
   def plot_smoothing_variation(corpus, k_values):
       # Tu implementación aquí
       pass
   ```
   - Analiza los resultados y discute cómo la variación de $k$ afecta las probabilidades suavizadas.

5. **Aplicación en clasificación de textos con N-Gramas:**
   - Implementa un clasificador de textos simple basado en n-gramas con suavizado **add-k**. Utiliza los trigramas de un corpus de textos para clasificar las oraciones:
   ```python
   def classify_texts_with_add_k(corpus, k):
       # Tu implementación aquí
       pass
   ```
   - Evalúa el rendimiento del clasificador con diferentes valores de $k$ y compara los resultados con un clasificador que utiliza suavizado de Laplace.
6. Después de completar los ejercicios prácticos, reflexiona sobre los resultados obtenidos. ¿Cuándo resulta más efectivo el suavizado **add-k** que el suavizado de Laplace? ¿Qué situaciones requieren ajustes más finos en la transferencia de masa de probabilidad a eventos no vistos?.

In [ ]:
## Tus respuestas

### Backoff y interpolación

El descuento que hemos estado discutiendo hasta ahora puede ayudar a resolver el problema de n-gramas de frecuencia cero. Pero hay una fuente adicional de conocimiento de la que podemos aprovechar. Si estamos tratando de calcular $P(w_n |w_{n-2} w_{n-1})$ pero no tenemos ejemplos de un trigram específico $w_{n-2} w_{n-1} w_n$, podemos estimar su probabilidad usando la probabilidad del bigrama $P(w_n|w_{n-1})$. De manera similar, si no tenemos conteos para calcular $P(w_n|w_{n-1})$, podemos recurrir al unigram $P(w_n)$.

En otras palabras, a veces usar menos contexto es algo bueno, ayudando a generalizar más para los contextos sobre los que el modelo no ha aprendido mucho. Hay dos formas de utilizar esta "jerarquía" de n-gramas. En el backoff, usamos el trigrama si la evidencia es suficiente, de lo contrario usamos el bigrama, y si no, el unigrama. En otras palabras, solo "retrocedemos" a un n-grama de menor orden si no tenemos evidencia de un n-grama de mayor orden. Por el contrario, en la interpolación, siempre combinamos las estimaciones de probabilidad de todos los estimadores de n-gramas, ponderando y combinando los conteos de trigramas, bigramas y unigrams.

En la interpolación lineal simple, combinamos n-gramas de diferentes órdenes interpolándolos linealmente. Así, estimamos la probabilidad del trigrama $P(w_n |w_{n-2} w_{n-1})$ mezclando las probabilidades de unigramas, bigramas y trigramas, cada una ponderada por un $\lambda$:

$$\hat{P}(w_n | w_{n-2}w_{n-1}) = \lambda_1 P(w_n) + \lambda_2 P(w_n | w_{n-1}) + \lambda_3 P(w_n | w_{n-2}w_{n-1})$$


Los $\lambda$ deben sumar `1`, haciendo que la ecuación sea equivalente a un promedio ponderado. En una versión ligeramente más sofisticada de la interpolación lineal, cada peso $\lambda$ se calcula condicionando en el contexto. De esta manera, si tenemos conteos particularmente precisos para un bigrama específico, asumimos que los conteos de los trigramas basados en este bigrama serán más confiables, por lo que podemos aumentar los $\lambda$ para esos trigramas y, por lo tanto, darle más peso al trigrama en la interpolación. La ecuación  muestra la ecuación para la interpolación con pesos condicionados por el contexto:

$$\hat{P}(w_n | w_{n-2}w_{n-1}) = \lambda_1 (w_{n-2}w_{n-1}) P(w_n) + \lambda_2 (w_{n-2}w_{n-1}) P(w_n | w_{n-1}) + \lambda_3 (w_{n-2}w_{n-1}) P(w_n | w_{n-2}w_{n-1})$$


¿Cómo se establecen estos valores de $\lambda$? 

Tanto la interpolación simple como la interpolación condicionada de $\lambda$ se aprenden a partir de un corpus reservado. Un corpus reservado es un corpus de entrenamiento adicional, llamado así porque lo reservamos del conjunto de entrenamiento, y lo usamos para establecer hiperparámetros como estos valores $\lambda$. Lo hacemos eligiendo los valores de $\lambda$ que maximizan la probabilidad del corpus reservado. Es decir, fijamos las probabilidades de n-gramas y luego buscamos los valores de $\lambda$ que, cuando se introducen la ecuaciones anteriores, nos dan la mayor probabilidad para el conjunto reservado. Hay varias formas de encontrar este conjunto óptimo de $\lambda$. Una forma es usar el algoritmo EM, un algoritmo de aprendizaje iterativo que converge en valores $\lambda$ localmente óptimos.

En un modelo de n-gramas backoff, si el n-grama que necesitamos tiene conteos cero, lo aproximamos retrocediendo al (n-1)-grama. Continuamos retrocediendo hasta llegar a un historial que tenga algunos conteos. Para que un modelo backoff proporcione una distribución de probabilidad correcta, tenemos que descontar los n-gramas de mayor orden para reservar algo de masa de probabilidad para los n-gramas de menor orden. Al igual que con el suavizado add-k, si los n-gramas de mayor orden no se descuentan y usamos simplemente la probabilidad MLE sin descuentos, entonces, tan pronto como reemplazáramos un n-grama con probabilidad cero por un n-grama de menor orden, estaríamos añadiendo masa de probabilidad, y la probabilidad total asignada a todas las cadenas posibles por el modelo de lenguaje sería mayor a 1. Además de este factor de descuento explícito, necesitaremos una función $\alpha$ para distribuir esta masa de probabilidad a los n-gramas de menor orden.

Este tipo de backoff con descuento también se llama backoff Katz. En el backoff de Katz confiamos en una probabilidad descontada $P^∗$ si hemos visto este n-grama antes (es decir, si tenemos conteos no nulos). De lo contrario, retrocedemos recursivamente a la probabilidad de Katz para el n-grama de historia más corta $(n-1)$. La probabilidad para un n-grama backoff $P_{BO}$ se calcula de la siguiente manera:

$$
P_{\text{BO}}(w_n | w_{n-N+1:n-1}) = 
\begin{cases} 
P^*(w_n | w_{n-N+1:n-1}), & \text{if } C(w_{n-N+1:n-1}) > 0 \\
\alpha(w_{n-N+1:n-1}) P_{\text{BO}}(w_n | w_{n-N+2:n-1}), & \text{en otros casos.}
\end{cases}
$$

El backoff de Katz a menudo se combina con un método de suavizado llamado Good-Turing. El algoritmo combinado de backoff Good-Turing implica cálculos bastante detallados para estimar el suavizado Good-Turing y los valores $P^*$ y $\alpha$.

### Ejemplos

#### 1. Backoff

**Descripción**: El enfoque de **backoff** retrocede a un n-grama de menor orden cuando no se encuentran suficientes evidencias para un n-grama de mayor orden. Por ejemplo, si no tenemos conteos para un trigram $P(w_n | w_{n-2} w_{n-1})$, retrocedemos al bigrama $P(w_n | w_{n-1})$. Si tampoco hay evidencia suficiente para el bigrama, retrocedemos al unigrama $P(w_n)$.

**Ejemplo**: 
Supongamos el siguiente corpus:
- "el gato duerme"
- "el perro ladra"

1. **Conteos de trigramas**:
   - $C(\text{"el gato duerme"}) = 1$
   - $C(\text{"gato duerme rápido"}) = 0$  (no existe)

2. **Probabilidad de trigramas**:
   - $P(\text{"duerme"} | \text{"el gato"})$ tiene un conteo, así que se utiliza:  
   $$
   P(\text{"duerme"} | \text{"el gato"}) = \frac{C(\text{"el gato duerme"})}{C(\text{"el gato"})} = \frac{1}{1} = 1
   $$

3. **Backoff**:
   - Si buscamos $P(\text{"rápido"} | \text{"gato duerme"})$, no tenemos el trigrama. Así que retrocedemos al bigrama:
   $$
   P(\text{"rápido"} | \text{"duerme"}) \approx P(\text{"rápido"})  \quad \text{(si tampoco hay bigrama)}
   $$

#### 2. Interpolación

**Descripción**: En la **interpolación**, en lugar de retroceder, combinamos las probabilidades de diferentes órdenes de n-gramas. Por ejemplo, en la interpolación lineal, ponderamos las probabilidades de unigramas, bigramas y trigramas para obtener una estimación más general.

**Ejemplo**:
Dado el corpus anterior y las siguientes probabilidades (supongamos suavizadas):
- $P(\text{"duerme"} | \text{"el gato"}) = 0.5$
- $P(\text{"duerme"} | \text{"gato"}) = 0.3$
- $P(\text{"duerme"}) = 0.2$

Supongamos los pesos $\lambda_1 = 0.1$, $\lambda_2 = 0.3$ y $\lambda_3 = 0.6$. La probabilidad interpolada es:

$$
\hat{P}(\text{"duerme"} | \text{"el gato"}) = \lambda_1 P(\text{"duerme"}) + \lambda_2 P(\text{"duerme"} | \text{"gato"}) + \lambda_3 P(\text{"duerme"} | \text{"el gato"})
$$

$$
\hat{P}(\text{"duerme"} | \text{"el gato"}) = 0.1 \times 0.2 + 0.3 \times 0.3 + 0.6 \times 0.5 = 0.02 + 0.09 + 0.3 = 0.41
$$

#### 3. Modelo de N-Gramas Backoff

**Descripción**: En un modelo de n-gramas backoff, retrocedemos de trigramas a bigramas y luego a unigramas si no tenemos suficiente evidencia para los n-gramas de mayor orden. Necesitamos un factor de descuento explícito para mantener la correcta distribución de probabilidad.

**Ejemplo**:
Supongamos que utilizamos el siguiente corpus:
- "el gato come"
- "el perro duerme"

Queremos calcular $P_{\text{BO}}(\text{"come"} | \text{"el gato"})$.

1. **Trigramas**:
   - $C(\text{"el gato come"}) = 1$

2. **Bigramas**:
   - $C(\text{"gato come"}) = 1$

3. **Probabilidad con Backoff**:
   - Si el trigram no existiera $( C(\text{"el gato come"}) = 0 $), usaríamos el bigrama:
   $$
   P_{\text{BO}}(\text{"come"} | \text{"gato"}) = \alpha P(\text{"come"} | \text{"gato"})
   $$

#### 4. Backoff de Katz

**Descripción**: El **backoff de Katz** utiliza una probabilidad descontada si el n-grama ha sido visto antes; si no, retrocede al n-grama de menor orden. Incluye un factor de descuento $\alpha$ para distribuir la masa de probabilidad correctamente.

**Ejemplo**:
Supongamos los siguientes conteos:
- $C(\text{"el gato come"}) = 1$
- $C(\text{"gato come"}) = 1$

La probabilidad para un n-grama backoff se calcula como:

$$
P_{\text{BO}}(\text{"come"} | \text{"el gato"}) = 
\begin{cases} 
P^*(\text{"come"} | \text{"el gato"}), & \text{si } C(\text{"el gato"}) > 0 \\
\alpha(\text{"el gato"}) P_{\text{BO}}(\text{"come"} | \text{"gato"}), & \text{en otros casos}
\end{cases}
$$

Supongamos que $\alpha(\text{"el gato"}) = 0.4$. Si no hay suficiente conteo para el trigrama:

$$
P_{\text{BO}}(\text{"come"} | \text{"el gato"}) = 0.4 \times P(\text{"come"} | \text{"gato"})
$$

#### 5: Good-Turing

**Descripción**: La estimación de **Good-Turing** ajusta los conteos de n-gramas observados para reservar masa de probabilidad a los n-gramas no vistos. Esta técnica redistribuye la probabilidad de manera que incluso las secuencias no observadas tengan una probabilidad distinta de cero.

#### Ejemplo Continuado:
Supongamos que tenemos los siguientes conteos de bigramas:
- $C(\text{"gato duerme"}) = 5$
- $C(\text{"perro duerme"}) = 1$
- $C(\text{"ratón duerme"}) = 0$

Queremos calcular las probabilidades ajustadas usando la estimación de **Good-Turing**.

1. **Paso 1: Calcular los números de conteo**:
   - Número de bigramas con conteo 1 ($ N_1$): Número de bigramas que ocurren exactamente una vez en el corpus. En este caso, $N_1 = 1$ ("perro duerme").
   - Número de bigramas con conteo 5 ($N_5$): Número de bigramas que ocurren exactamente cinco veces en el corpus. Aquí, $N_5 = 1$ ("gato duerme").

2. **Paso 2: Aplicar la estimación de Good-Turing**:
   - Para bigramas con conteo mayor a cero, ajustamos los conteos:
     - Para "gato duerme" ($C = 5$):
       $$
       P^*(\text{"gato duerme"}) = \frac{(C + 1) \times \frac{N_{C+1}}{N_C}}{N}
       $$
       Como no hay bigramas con \( C+1 = 6 \), podemos aproximar esta fracción con \( \frac{N_5}{N_4} \approx \frac{N_1}{N_5} \):
       $$
       P^*(\text{"gato duerme"}) \approx \frac{(5 + 1) \times \frac{N_1}{N_5}}{N} = \frac{6 \times \frac{1}{1}}{N} = \frac{6}{N}
       $$
     - Normalizamos para obtener la probabilidad:
       $$
       P^*(\text{"gato duerme"}) = \frac{6}{N}
       $$

   - Para "ratón duerme" ($C = 0$):
     - Redistribuimos la masa de probabilidad:
       $$
       P^*(\text{"ratón duerme"}) = \frac{N_1}{N}
       $$

3. **Paso 3: Probabilidad ajustada para "gato duerme" y "ratón duerme"**:
   - $N$: Total de bigramas en el corpus. Supongamos $N = 10$ (suma de todos los bigramas).
   - Probabilidad ajustada para "gato duerme":
     $$
     P^*(\text{"gato duerme"}) = \frac{6}{10} = 0.6
     $$
   - Probabilidad ajustada para "ratón duerme" (evento no visto):
     $$
     P^*(\text{"ratón duerme"}) = \frac{1}{10} = 0.1
     $$

### 6. Algoritmo combinado de Backoff Good-Turing

**Descripción**: Este algoritmo combina el backoff de Katz con la estimación de Good-Turing. Primero, se aplican los descuentos de Good-Turing a los n-gramas observados, y luego se utiliza el backoff para distribuir la masa de probabilidad restante a los n-gramas de menor orden.

#### Ejemplo:
Queremos calcular $P_{\text{BO}}(\text{"juega"} | \text{"el gato"})$ utilizando el backoff de Katz y el suavizado de Good-Turing.

1. **Paso 1: Estimar conteos con Good-Turing**:
   - Supongamos que los conteos ajustados de Good-Turing para algunos bigramas son:
     - $C^*(\text{"el gato"}) = 3.5$
     - $C^*(\text{"gato duerme"}) = 2.0$
     - $C^*(\text{"gato juega"}) = 0.0$ (no observado)

2. **Paso 2: Calcular la probabilidad ajustada**:
   - Para $P^*(\text{"duerme"} | \text{"el gato"})$:
     $$
     P^*(\text{"duerme"} | \text{"el gato"}) = \frac{C^*(\text{"el gato duerme"})}{C^*(\text{"el gato"})} = \frac{2.0}{3.5} \approx 0.571
     $$

   - Para "gato juega", como $C^*(\text{"gato juega"}) = 0$:
     - Utilizamos backoff y asignamos una probabilidad:
       $$
       P_{\text{BO}}(\text{"juega"} | \text{"gato"}) = \alpha(\text{"gato"}) P^*(\text{"juega"} | \text{"gato"})
       $$

3. **Paso 3: Backoff**:
   - Asumamos que el factor de backoff $\alpha(\text{"gato"}) = 0.4$:
     $$
     P_{\text{BO}}(\text{"juega"} | \text{"gato"}) = 0.4 \times P(\text{"juega"})
     $$

4. **Paso 4: Probabilidad final**:
   - Combinamos las probabilidades ajustadas y backoff:
     $$
     P_{\text{BO}}(\text{"juega"} | \text{"el gato"}) = 0.4 \times P(\text{"juega"})
     $$

Este método asegura que las probabilidades se redistribuyan correctamente, incluso cuando no hay suficientes datos para los n-gramas de mayor orden.


#### Ejemplo con un corpus más grande

#### Corpus:
Supongamos el siguiente corpus, con 8 oraciones para mayor complejidad:

1. "el gato duerme"
2. "el gato come pescado"
3. "el perro ladra"
4. "el perro duerme"
5. "el gato corre"
6. "el perro come carne"
7. "el ratón come queso"
8. "el gato y el perro juegan"

Queremos calcular la probabilidad de la secuencia "el gato come" utilizando interpolación y backoff de Katz.

#### Conteos de N-gramas

1. **Unigramas** (frecuencia de palabras individuales):
   - $C(\text{"el"}) = 8$
   - $C(\text{"gato"}) = 4$
   - $C(\text{"duerme"}) = 2$
   - $C(\text{"come"}) = 3$
   - $C(\text{"perro"}) = 4$
   - $C(\text{"pescado"}) = 1$
   - $C(\text{"carne"}) = 1$
   - $C(\text{"ratón"}) = 1$
   - $C(\text{"queso"}) = 1$
   - $C(\text{"corre"}) = 1$
   - $C(\text{"ladra"}) = 1$
   - $C(\text{"y"}) = 1$
   - $C(\text{"juegan"}) = 1$

2. **Bigramas** (frecuencia de pares de palabras):
   - $C(\text{"el gato"}) = 4$
   - $C(\text{"gato duerme"}) = 1$
   - $C(\text{"gato come"}) = 1$
   - $C(\text{"el perro"}) = 4$
   - $C(\text{"perro ladra"}) = 1$
   - $C(\text{"perro duerme"}) = 1$
   - $C(\text{"perro come"}) = 1$
   - $C(\text{"ratón come"}) = 1$
   - $C(\text{"come pescado"}) = 1$
   - $C(\text{"come carne"}) = 1$
   - $C(\text{"come queso"}) = 1$
   - $C(\text{"gato corre"}) = 1$
   - $C(\text{"gato y"}) = 1$
   - $C(\text{"y el"}) = 1$
   - $C(\text{"el perro"}) = 2$
   - $C(\text{"perro juegan"}) = 1$

3. **Trigramas** (frecuencia de secuencias de tres palabras):
   - $C(\text{"el gato duerme"}) = 1$
   - $C(\text{"el gato come"}) = 1$
   - $C(\text{"gato come pescado"}) = 1$
   - $C(\text{"el perro ladra"}) = 1$
   - $C(\text{"el perro duerme"}) = 1$
   - $C(\text{"perro come carne"}) = 1$
   - $C(\text{"ratón come queso"}) = 1$
   - $C(\text{"el gato corre"}) = 1$
   - $C(\text{"el gato y"}) = 1$
   - $C(\text{"gato y el"}) = 1$
   - $C(\text{"y el perro"}) = 1$
   - $C(\text{"el perro juegan"}) = 1$

### 1. Interpolación lineal para el trigrama "el gato come"

Queremos calcular la probabilidad interpolada de $P(\text{"come"} | \text{"el gato"})$ utilizando interpolación lineal con $\lambda_1 = 0.2$, $\lambda_2 = 0.3$, $\lambda_3 = 0.5$:

1. **Probabilidades MLE**:
   - Unigrama: 
     $$
     P(\text{"come"}) = \frac{C(\text{"come"})}{\text{total de unigramas}} = \frac{3}{24} = 0.125
     $$
   - Bigrama:
     $$
     P(\text{"come"} | \text{"gato"}) = \frac{C(\text{"gato come"})}{C(\text{"gato"})} = \frac{1}{4} = 0.25
     $$
   - Trigrama:
     $$
     P(\text{"come"} | \text{"el gato"}) = \frac{C(\text{"el gato come"})}{C(\text{"el gato"})} = \frac{1}{4} = 0.25
     $$

2. **Probabilidad interpolada**:
   $$
   \hat{P}(\text{"come"} | \text{"el gato"}) = \lambda_1 P(\text{"come"}) + \lambda_2 P(\text{"come"} | \text{"gato"}) + \lambda_3 P(\text{"come"} | \text{"el gato"})
   $$

   $$
   \hat{P}(\text{"come"} | \text{"el gato"}) = 0.2 \times 0.125 + 0.3 \times 0.25 + 0.5 \times 0.25
   $$

   $$
   \hat{P}(\text{"come"} | \text{"el gato"}) = 0.025 + 0.075 + 0.125 = 0.225
   $$

#### 2. Backoff de Katz para el trigrama "el gato corre"

Supongamos que queremos calcular $P_{\text{BO}}(\text{"corre"} | \text{"el gato"})$ usando backoff de Katz:

1. **Conteos**:
   - Trigrama: $C(\text{"el gato corre"}) = 1$
   - Bigrama: $C(\text{"gato corre"}) = 1$

2. **Descuento**:
   - Para trigramas con conteos, usamos $P^*(w_n | w_{n-2} w_{n-1})$. Supongamos que aplicamos un descuento $D = 0.75$:
   $$
   P^*(\text{"corre"} | \text{"el gato"}) = \frac{C(\text{"el gato corre"}) - D}{C(\text{"el gato"})}
   $$

   $$
   P^*(\text{"corre"} | \text{"el gato"}) = \frac{1 - 0.75}{4} = \frac{0.25}{4} = 0.0625
   $$

3. **Backoff**:
   - Si el trigrama no tuviera suficiente conteo, usaríamos el bigrama y aplicamos el factor $\alpha$:
   $$
   P_{\text{BO}}(\text{"corre"} | \text{"el gato"}) = \alpha(\text{"el gato"}) P(\text{"corre"} | \text{"gato"})
   $$

4. **Probabilidad final**:
   - Dado que $C(\text{"el gato corre"}) > 0$:
   $$
   P_{\text{BO}}(\text{"corre"} | \text{"el gato"}) = P^*(\text{"corre"} | \text{"el gato"}) = 0.0625
   $$


- **Interpolación**: Nos permite combinar información de n-gramas de diferentes órdenes para una estimación más robusta. En este caso, la probabilidad interpolada de "el gato come" fue $0.225$, al incorporar información de unigramas, bigramas y trigramas.
- **Backoff de Katz**: Proporciona una forma de retroceder a n-gramas de menor orden cuando los conteos de n-gramas de mayor orden son insuficientes. En este caso, se utilizó un factor de descuento para ajustar la probabilidad del trigrama "el gato corre".


### Implementaciones

#### 1. Backoff

**Descripción**: El método **backoff** retrocede a un n-grama de menor orden cuando no se encuentran suficientes evidencias para un n-grama de mayor orden. Si no tenemos un trigram $P(w_n | w_{n-2} w_{n-1})$, retrocedemos al bigrama $P(w_n | w_{n-1})$. Si tampoco existe, retrocedemos al unigrama $P(w_n)$.

**Ecuación**:

$$
P_{\text{BO}}(w_n | w_{n-2} w_{n-1}) = 
\begin{cases} 
P^*(w_n | w_{n-2} w_{n-1}), & \text{si } C(w_{n-2} w_{n-1} w_n) > 0 \\
\alpha(w_{n-2} w_{n-1}) P_{\text{BO}}(w_n | w_{n-1}), & \text{si no}
\end{cases}
$$

In [ ]:
def backoff_unigram(unigram_counts, total_tokens):
    # Calcula las probabilidades de unigramas
    unigram_probs = {}
    for word, count in unigram_counts.items():
        unigram_probs[word] = count / total_tokens
    return unigram_probs

def backoff_bigram(bigram_counts, unigram_counts, total_tokens):
    # Calcula las probabilidades de bigramas con retroceso a unigramas
    bigram_probs = {}
    for (w1, w2), count in bigram_counts.items():
        if unigram_counts[w1] > 0:
            bigram_probs[(w1, w2)] = count / unigram_counts[w1]
        else:
            bigram_probs[(w1, w2)] = backoff_unigram(unigram_counts, total_tokens).get(w2, 1 / total_tokens)
    return bigram_probs


#### 2. Interpolación

**Descripción**: La **interpolación** combina las probabilidades de diferentes órdenes de n-gramas (unigramas, bigramas, trigramas) para obtener una estimación más general. La interpolación lineal pondera estas probabilidades mediante los parámetros $\lambda$ que deben sumar 1.

**Ecuación**:

$$
\hat{P}(w_n | w_{n-2}w_{n-1}) = \lambda_1 P(w_n) + \lambda_2 P(w_n | w_{n-1}) + \lambda_3 P(w_n | w_{n-2}w_{n-1})
$$

In [ ]:
def interpolation(unigram_probs, bigram_probs, trigram_probs, lambda1, lambda2, lambda3):
    def interpolated_prob(w_n, w_n1, w_n2):
        # Calcula la probabilidad interpolada
        p_unigram = unigram_probs.get(w_n, 0)
        p_bigram = bigram_probs.get((w_n1, w_n), 0)
        p_trigram = trigram_probs.get((w_n2, w_n1, w_n), 0)
        return lambda1 * p_unigram + lambda2 * p_bigram + lambda3 * p_trigram
    return interpolated_prob

# Ejemplo de uso
unigram_probs = {'gato': 0.2, 'duerme': 0.1}
bigram_probs = {('el', 'gato'): 0.5, ('gato', 'duerme'): 0.3}
trigram_probs = {('el', 'gato', 'duerme'): 0.6}
lambda1, lambda2, lambda3 = 0.1, 0.3, 0.6
interpolated_prob = interpolation(unigram_probs, bigram_probs, trigram_probs, lambda1, lambda2, lambda3)
print(interpolated_prob('duerme', 'gato', 'el'))  # Resultado interpolado

#### 3. Modelo de N-Gramas Backoff

**Descripción**: En un modelo de n-gramas backoff, retrocedemos de trigramas a bigramas y luego a unigramas si no tenemos suficiente evidencia para los n-gramas de mayor orden. Debemos descontar los n-gramas de mayor orden para reservar masa de probabilidad para los n-gramas de menor orden.

**Ecuación**:
$$
P_{\text{BO}}(w_n | w_{n-2} w_{n-1}) = 
\begin{cases} 
P^*(w_n | w_{n-2} w_{n-1}), & \text{si } C(w_{n-2} w_{n-1} w_n) > 0 \\
\alpha(w_{n-2} w_{n-1}) P_{\text{BO}}(w_n | w_{n-1}), & \text{si no}
\end{cases}
$$

In [ ]:
def ngram_backoff(unigram_probs, bigram_probs, trigram_probs, alpha):
    def backoff_prob(w_n, w_n1, w_n2):
        # Trigrama
        if (w_n2, w_n1, w_n) in trigram_probs:
            return trigram_probs[(w_n2, w_n1, w_n)]
        # Bigram
        elif (w_n1, w_n) in bigram_probs:
            return alpha * bigram_probs[(w_n1, w_n)]
        # Unigram
        else:
            return alpha * unigram_probs.get(w_n, 1 / len(unigram_probs))
    return backoff_prob

# Ejemplo de uso
alpha = 0.4
backoff_prob = ngram_backoff(unigram_probs, bigram_probs, trigram_probs, alpha)
print(backoff_prob('duerme', 'gato', 'el'))  # Resultado con retroceso


#### 4. Backoff de Katz

**Descripción**: El **backoff de Katz** usa un factor de descuento explícito y una función de backoff $\alpha$ para reasignar masa de probabilidad a los n-gramas de menor orden cuando los n-gramas de mayor orden no tienen suficiente conteo.

**Ecuación**:
$$
P_{\text{BO}}(w_n | w_{n-2} w_{n-1}) = 
\begin{cases} 
P^*(w_n | w_{n-2} w_{n-1}), & \text{si } C(w_{n-2} w_{n-1} w_n) > 0 \\
\alpha(w_{n-2} w_{n-1}) P_{\text{BO}}(w_n | w_{n-1}), & \text{si no}
\end{cases}
$$

In [ ]:
def katz_backoff(unigram_counts, bigram_counts, trigram_counts, D=0.75):
    def discounted_prob(count, context_count):
        return max(count - D, 0) / context_count

    def backoff_prob(w_n, w_n1, w_n2):
        # Trigrama
        trigram = (w_n2, w_n1, w_n)
        if trigram in trigram_counts:
            count = trigram_counts[trigram]
            context_count = bigram_counts.get((w_n2, w_n1), 1)
            return discounted_prob(count, context_count)
        # Bigram
        bigram = (w_n1, w_n)
        if bigram in bigram_counts:
            count = bigram_counts[bigram]
            context_count = unigram_counts.get(w_n1, 1)
            return discounted_prob(count, context_count)
        # Unigram
        return unigram_counts.get(w_n, 1) / sum(unigram_counts.values())

    return backoff_prob

# Ejemplo de uso
unigram_counts = {'gato': 5, 'duerme': 3}
bigram_counts = {('el', 'gato'): 4, ('gato', 'duerme'): 2}
trigram_counts = {('el', 'gato', 'duerme'): 1}
katz_prob = katz_backoff(unigram_counts, bigram_counts, trigram_counts)
print(katz_prob('duerme', 'gato', 'el'))  # Resultado con backoff de Katz


#### 5. Good-Turing

**Descripción**: La estimación **Good-Turing** ajusta los conteos observados de n-gramas para reasignar masa de probabilidad a los eventos no observados.

**Ecuación**:
$$
P^*(w_n | w_{n-2} w_{n-1}) = \frac{(C + 1) \times \frac{N_{C+1}}{N_C}}{N}
$$

In [ ]:
from collections import Counter

def good_turing_counts(ngram_counts):
    count_of_counts = Counter(ngram_counts.values())
    adjusted_counts = {}
    
    for ngram, count in ngram_counts.items():
        if count + 1 in count_of_counts:
            adjusted_counts[ngram] = (count + 1) * (count_of_counts[count + 1] / count_of_counts[count])
        else:
            adjusted_counts[ngram] = count

    return adjusted_counts

# Ejemplo de uso
ngram_counts = {'el gato duerme': 5, 'perro duerme': 1, 'ratón duerme': 0}
adjusted_counts = good_turing_counts(ngram_counts)
print(adjusted_counts)  # Conteos ajustados con Good-Turing


#### 6. Algoritmo combinado de backoff Good-Turing

**Descripción**: Este algoritmo combina el backoff de Katz con la estimación de Good-Turing. Descuenta los n-gramas y distribuye la masa de probabilidad usando Good-Turing.


In [ ]:
def backoff_good_turing(unigram_counts, bigram_counts, trigram_counts, D=0.75):
    adjusted_counts = good_turing_counts(trigram_counts)

    def backoff_prob(w_n, w_n1, w_n2):
        trigram = (w_n2, w_n1, w_n)
        if trigram in adjusted_counts:
            count = adjusted_counts[trigram]
            context_count = bigram_counts.get((w_n2, w_n1), 1)
            return count / context_count
        return katz_backoff(unigram_counts, bigram_counts, adjusted_counts)(w_n, w_n1, w_n2)

    return backoff_prob

# Ejemplo de uso
backoff_good_turing_prob = backoff_good_turing(unigram_counts, bigram_counts, trigram_counts)
print(backoff_good_turing_prob('duerme', 'gato', 'el'))  # Resultado con backoff Good-Turing


### Ejercicios

#### **Ejercicio 1: Análisis de backoff en un corpus grande**

Dado un corpus extenso con los siguientes conteos:

- **Unigramas**:
  - $C(\text{"el"}) = 1000$
  - $C(\text{"gato"}) = 500$
  - $C(\text{"duerme"}) = 200$

- **Bigramas**:
  - $C(\text{"el gato"}) = 300$
  - $C(\text{"gato duerme"}) = 100$

- **Trigramas**:
  - $C(\text{"el gato duerme"}) = 50$

1. Calcula la probabilidad del trigram "el gato duerme" utilizando **backoff** si el trigrama "el gato duerme" no existiera en el corpus. 
2. Explica los pasos para calcular esta probabilidad, retrocediendo a bigramas y luego a unigramas si es necesario.
3. Analiza las ventajas y desventajas de utilizar backoff en lugar de depender exclusivamente de n-gramas de mayor orden.

#### **Ejercicio 2: Optimización de pesos en interpolación**

Suponga que tiene los siguientes valores de probabilidad suavizada para un trigrama:

- $P(\text{"gato"}) = 0.2$ (Unigrama)
- $P(\text{"duerme"} | \text{"gato"}) = 0.4$ (Bigrama)
- $P(\text{"duerme"} | \text{"el gato"}) = 0.6$ (Trigrama)

Los pesos iniciales para la interpolación lineal son $\lambda_1 = 0.2$, $\lambda_2 = 0.3$, $\lambda_3 = 0.5$.

1. Determina la probabilidad interpolada $\hat{P}(\text{"duerme"} | \text{"el gato"})$ con los pesos dados.
2. Proporciona una estrategia para optimizar estos pesos $\lambda$ utilizando un conjunto de validación con el objetivo de maximizar la probabilidad total del corpus.
3. Explica cómo el algoritmo EM podría ser utilizado para ajustar iterativamente estos pesos de interpolación.

#### **Ejercicio 3: Implementación de descuentos en backoff de Katz**

Dado el siguiente conjunto de datos:

- **Trigramas**:
  - $C(\text{"el gato duerme"}) = 2$
- **Bigramas**:
  - $C(\text{"gato duerme"}) = 5$
- **Unigramas**:
  - $C(\text{"duerme"}) = 10$

1. Define cómo calcular el descuento para el trigram "el gato duerme" usando el backoff de Katz.
2. Determina la probabilidad descontada \( P^*(\text{"duerme"} | \text{"el gato"}) \) para este trigram.
3. Analiza cómo el factor de descuento afecta la distribución de probabilidad y qué sucede con los n-gramas de menor orden cuando se aplica backoff de Katz.


#### **Ejercicio 4: Ajuste de conteos con Good-Turing**

Considera el siguiente conjunto de conteos para un corpus grande:

- $C(\text{"gato duerme"}) = 4$
- $C(\text{"perro duerme"}) = 2$
- $C(\text{"ratón duerme"}) = 0$
- Número de bigramas que ocurren exactamente una vez $( N_1) = 3$
- Número de bigramas que ocurren exactamente dos veces $( N_2 ) = 2$.

1. Calcula la probabilidad ajustada usando **Good-Turing** para el bigrama "gato duerme".
2. Explica cómo Good-Turing reasigna la masa de probabilidad a los eventos no observados y por qué esto es útil en el modelado de lenguajes.
3. Determina la probabilidad para un bigrama no observado como "ratón duerme" y justifique cómo Good-Turing evita que se asigne una probabilidad de cero a estos eventos.

#### **Ejercicio 5: Análisis comparativo de backoff Good-Turing**

Suponga que está trabajando con un corpus de entrenamiento y un conjunto de desarrollo separado. El modelo utiliza el **algoritmo combinado de Backoff Good-Turing** para calcular las probabilidades de n-gramas.

1. Describe el proceso completo para calcular la probabilidad de un n-grama utilizando el algoritmo combinado de Backoff Good-Turing.
2. Compara el rendimiento de este modelo con un modelo de n-grama simple que solo utilice interpolación. ¿En qué situaciones el algoritmo combinado proporciona una ventaja significativa?
3. Propone una estrategia para evaluar la precisión del modelo en un conjunto de prueba y cómo ajustar los parámetros (como el factor de descuento $D$ para mejorar el rendimiento general.

#### **Ejercicio 6: Aplicación práctica en clasificación de textos**

Suponga que está diseñando un sistema de clasificación de textos que utiliza n-gramas para representar documentos. Decide implementar un modelo que combine **backoff** e **interpolación**.

1. Explica cómo utilizaría las técnicas de backoff y interpolación para calcular las probabilidades de n-gramas en este contexto.
2. Describe un procedimiento para ajustar los pesos de interpolación y los factores de descuento de backoff en función de un conjunto de datos de entrenamiento.
3. Analiza los posibles desafíos y soluciones cuando se trabaja con un vocabulario grande y datos desbalanceados en la clasificación de textos.


#### **Ejercicio 7: Evaluación del impacto de backoff en la generalización del modelo**

Se te proporciona un corpus con las siguientes características:

- El corpus contiene 1 millón de palabras.
- Hay un conjunto significativo de trigramas y bigramas raros (con conteos menores o iguales a 2).

1. Explica cómo el uso del **backoff** puede ayudar a mejorar la generalización del modelo en presencia de estos n-gramas raros.
2. Propone una métrica para evaluar el impacto del backoff en la generalización del modelo, particularmente en un conjunto de prueba que contiene contextos no vistos.
3. Realiza un análisis teórico sobre cómo la reducción del contexto (retrocediendo a bigramas y unigramas) afecta la precisión del modelo en situaciones con datos escasos.

#### **Ejercicio 8: Ajuste dinámico de pesos en interpolación condicionada**

Imagina que estás desarrollando un modelo de lenguaje que necesita ajustar dinámicamente los pesos $\lambda$ en la interpolación en función del contexto.

1. Diseña un esquema para ajustar dinámicamente los pesos de interpolación $\lambda$ basados en la precisión de los conteos en diferentes contextos (unigrama, bigrama, trigrama).
2. Describe cómo utilizarías un conjunto de validación para aprender estos pesos condicionales y qué técnica utilizarías para evitar el sobreajuste.
3. Analiza los pros y los contras de utilizar interpolación simple frente a interpolación condicionada en escenarios donde los contextos varían significativamente en términos de frecuencia.

#### **Ejercicio 9: Análisis de la masa de probabilidad en backoff de Katz**

Dado un modelo que utiliza el **backoff de Katz** con un factor de descuento $D = 0.75$:

1. Explica cómo se redistribuye la masa de probabilidad cuando se encuentra un n-grama con conteo cero.
2. Dado un contexto específico donde se retrocede a un bigrama y luego a un unigrama, analice cómo se distribuye la probabilidad entre los diferentes órdenes de n-gramas.
3. Proporciona una estrategia para ajustar el factor de descuento $D$ en función del tamaño del corpus y la frecuencia de los n-gramas de mayor orden.

#### **Ejercicio 10: Evaluación del rendimiento de Good-Turing en corpus multilingüe**

Tienes un corpus multilingüe con textos en inglés, español y francés. Cada idioma tiene un conjunto distinto de unigramas y bigramas con diferentes frecuencias.

1. Explica cómo el método de **Good-Turing** puede utilizarse para ajustar las probabilidades de n-gramas en este corpus multilingüe.
2. Analiza cómo la variación en la distribución de frecuencias entre los diferentes idiomas puede afectar la estimación de Good-Turing y cómo manejarías esta variabilidad.
3. Diseña un experimento para evaluar si Good-Turing mejora la precisión de un modelo de lenguaje en cada idioma individualmente y en un corpus combinado. ¿Qué métricas utilizarías para esta evaluación?

#### **Ejercicio 11: Construcción de un modelo de lenguaje robustecido con Backoff Good-Turing**

Estás desarrollando un modelo de lenguaje que necesita ser robusto frente a la presencia de múltiples formas de un mismo n-grama (variaciones sintácticas).

1. Describe cómo implementarías un modelo de lenguaje que utilice el **algoritmo combinado de Backoff Good-Turing** para manejar estas variaciones de n-gramas.
2. Explica cómo equilibrarías el uso de backoff y Good-Turing para garantizar que se asigne masa de probabilidad suficiente a las variaciones no observadas de los n-gramas.
3. Analiza el rendimiento esperado del modelo en comparación con un modelo estándar de trigramas sin suavizado, particularmente en un corpus donde las variaciones son frecuentes.

#### **Ejercicio 12: Interpolación y suavizado en modelos de lenguaje contextuales**

Supón que estás trabajando con un modelo de lenguaje contextual que necesita combinar la información de n-gramas con características contextuales adicionales (como la posición en la oración).

1. Diseña un enfoque que combine la **interpolación** de n-gramas con características contextuales adicionales, explicando cómo integrarías estas características en la función de interpolación.
2. Proporciona un análisis sobre cómo las características contextuales pueden influir en los pesos de interpolación $\lambda$ y cómo ajustarías estos pesos utilizando un conjunto de desarrollo.
3. Discute los desafíos y beneficios de combinar interpolación de n-gramas con otras características contextuales en modelos de lenguaje modernos.


#### **Ejercicio 13: Exploración de modelos Backoff en sistemas de reconocimiento de voz**

Estás implementando un sistema de reconocimiento de voz que utiliza un modelo de lenguaje basado en trigramas para mejorar la precisión del reconocimiento.

1. Explica cómo implementarías el **backoff de Katz** para gestionar la alta variabilidad en las secuencias de palabras reconocidas.
2. Propone un método para ajustar el factor de backoff $\alpha$ en función de la calidad del reconocimiento de voz en tiempo real.
3. Analiza cómo la precisión del modelo de lenguaje backoff puede afectar la tasa de error de reconocimiento y cómo podrías mejorarla mediante un ajuste fino de los factores de descuento y backoff.


In [ ]:
## Tus respuestas